# Familial Model Using Single Batch 1 + Batch 2

In [1]:
import ktrain
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from ktrain import text
import random
import warnings
from sklearn.utils import shuffle

# Set random seed
random.seed(18)
seed = 18

# Ignore warnings
warnings.filterwarnings('ignore')

# Display options
pd.set_option('display.max_colwidth', None)

## 1. Loading the data and quick exploratory data analysis

In [25]:
import os

# Define the folder path and themes
folder_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/single_theme_using_jaccard_method'
themes = [
    'Aspirational', 'Attainment', 'Community Consciousness', 'Familial', 'Filial Piety', 
    'First Gen', 'Navigational', 'Perseverance', 'Resistance', 'Social', 'Spiritual'
]

# Initialize an empty dictionary to store DataFrames
theme_dataframes = {}
# Loop through each theme and load its corresponding file
for theme in themes:
    # Construct the filename without modifying the theme name
    file_name = f"{theme}_sentence_level_batch_1_jaccard.csv"
    file_path = os.path.join(folder_path, file_name)
    
    # Check if the file exists before attempting to load
    if os.path.exists(file_path):
        theme_dataframes[theme] = pd.read_csv(file_path)
        print(f"Loaded {file_name}")
    else:
        print(f"File not found for theme: {theme}")

Loaded Aspirational_sentence_level_batch_1_jaccard.csv
Loaded Attainment_sentence_level_batch_1_jaccard.csv
Loaded Community Consciousness_sentence_level_batch_1_jaccard.csv
Loaded Familial_sentence_level_batch_1_jaccard.csv
Loaded Filial Piety_sentence_level_batch_1_jaccard.csv
Loaded First Gen_sentence_level_batch_1_jaccard.csv
Loaded Navigational_sentence_level_batch_1_jaccard.csv
Loaded Perseverance_sentence_level_batch_1_jaccard.csv
Loaded Resistance_sentence_level_batch_1_jaccard.csv
Loaded Social_sentence_level_batch_1_jaccard.csv
Loaded Spiritual_sentence_level_batch_1_jaccard.csv


In [26]:
familial_df = theme_dataframes["Familial"]
filial_piety_df = theme_dataframes["Filial Piety"]

familial_df = pd.concat([familial_df, filial_piety_df])

familial_df.shape

(1299, 3)

In [29]:
# familial_df = theme_dataframes["Familial"]

# # Train-test split 
training_df, test_df = train_test_split(familial_df, test_size=0.2, random_state=42, stratify=familial_df['label'])

training_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [6]:
merged_familial_df_batch_1 = pd.read_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/merged_themes_using_jaccard_method/merged_Familial_sentence_level_batch_1_jaccard.csv", encoding='utf-8')
merged_familial_df_batch_2 = pd.read_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/merged_themes_using_jaccard_method/Familial Plus_sentence_level_batch_2_jaccard.csv", encoding='utf-8')

merged_familial_df = pd.concat([merged_familial_df_batch_1, merged_familial_df_batch_2])

# Shuffle the merged dataset
merged_familial_df = shuffle(merged_familial_df, random_state=seed)

# Train-test split 
training_df, test_df = train_test_split(merged_familial_df, test_size=0.3, random_state=42, stratify=merged_familial_df['label'])

training_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [30]:
print(f"Training dataset shape: {training_df.shape} \nTest dataset shape: {test_df.shape}")
pos_labels = len([n for n in training_df['label'] if n==1])
print("Positive labels present in the dataset : {}  out of {} or {}%".format(pos_labels, len(training_df['label']), (pos_labels/len(training_df['label']))*100))
pos_labels = len([n for n in test_df['label'] if n==1])
print("Positive labels present in the test dataset : {}  out of {} or {}%".format(pos_labels, len(test_df['label']), (pos_labels/len(test_df['label']))*100))

Training dataset shape: (1039, 3) 
Test dataset shape: (260, 3)
Positive labels present in the dataset : 109  out of 1039 or 10.490856592877767%
Positive labels present in the test dataset : 27  out of 260 or 10.384615384615385%


In [31]:
print(training_df.shape)
print(test_df.shape)

(1039, 3)
(260, 3)


## 2. Experimental Design

In [32]:
MAXLEN = 150

In [33]:
X = training_df['sentence']
y = training_df['label']

In [34]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 18, stratify=y)
X_test.shape

(104,)

In [35]:
model_name = 'bert-base-uncased'

In [36]:
distillbert_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0,1])
training_set = distillbert_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
validation_set = distillbert_transformer.preprocess_test(X_test.tolist(), y_test.tolist())
distillbert_base_model = distillbert_transformer.get_classifier()

preprocessing train...
language: en
train sequence lengths:
	mean : 20
	95percentile : 41
	99percentile : 58


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 20
	95percentile : 40
	99percentile : 53


In [37]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Define classes and class labels
classes = np.array([0, 1])
class_labels = list(training_df.label)

# Compute class weights

class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=class_labels)

# Print class weights
print(class_weights)

class_weights = dict(zip(classes, class_weights))

[0.55860215 4.76605505]


In [38]:
# Build BERT model
# model = text.text_classifier('distilbert', train_data=(X_train, y_train), preproc=distillbert_transformer)
distillbert_learner = ktrain.get_learner(distillbert_base_model, train_data=training_set, val_data=validation_set, batch_size=6)
# learner.fit_onecycle(2e-5, 4, class_weight=class_weights)
# learner.autofit(2.27E-06, early_stopping=4)
distillbert_learner.set_weight_decay(0.001)
distillbert_learner.autofit(4.96E-05, epochs=11, early_stopping=4, class_weight=class_weights)
# distillbert_learner.set_weight_decay(0.001)
# distillbert_learner.autofit(2.27E-06, early_stopping=4, class_weight=class_weights)



begin training using triangular learning rate policy with max lr of 4.96e-05...
Epoch 1/11
156/156 [==============================] - 86s 429ms/step - loss: 0.6576 - accuracy: 0.6920 - val_loss: 0.4729 - val_accuracy: 0.9038
Epoch 2/11
156/156 [==============================] - 54s 340ms/step - loss: 0.3988 - accuracy: 0.8107 - val_loss: 0.2343 - val_accuracy: 0.8942
Epoch 3/11
156/156 [==============================] - 51s 321ms/step - loss: 0.3638 - accuracy: 0.8086 - val_loss: 0.3050 - val_accuracy: 0.8846
Epoch 4/11
156/156 [==============================] - 52s 327ms/step - loss: 0.3191 - accuracy: 0.8374 - val_loss: 0.2263 - val_accuracy: 0.8942
Epoch 5/11
156/156 [==============================] - 52s 332ms/step - loss: 0.3149 - accuracy: 0.8588 - val_loss: 0.2656 - val_accuracy: 0.8942
Epoch 6/11
156/156 [==============================] - 53s 333ms/step - loss: 0.2882 - accuracy: 0.8642 - val_loss: 0.3246 - val_accuracy: 0.8750
Epoch 7/11
156/156 [============================

In [39]:
distillbert_learner.validate(class_names=distillbert_transformer.get_classes())

4/4 [==============================] - 10s 1s/step
              precision    recall  f1-score   support

           0       0.97      0.91      0.94        93
           1       0.50      0.73      0.59        11

    accuracy                           0.89       104
   macro avg       0.73      0.82      0.77       104
weighted avg       0.92      0.89      0.90       104



array([[85,  8],
       [ 3,  8]])

In [40]:
distillbert_learner.model.summary()

Model: "tf_bert_for_sequence_classification_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_151 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [41]:
distillbert_predictor = ktrain.get_predictor(distillbert_learner.model, preproc=distillbert_transformer)

In [42]:
distillbert_test_data = test_df['sentence'].tolist()
distillbert_test_label = test_df['label'].tolist()

In [43]:
y_pred_distillbert = distillbert_predictor.predict(distillbert_test_data)

In [44]:
y_pred_distillbert = [int(x) for x in y_pred_distillbert]

In [45]:
tn, fp, fn, tp = confusion_matrix(distillbert_test_label, y_pred_distillbert).ravel()
print('True Negative: {}, False Positive: {}, False Negative: {}, True Positive: {}'.format(tn, fp, fn, tp))

True Negative: 203, False Positive: 30, False Negative: 7, True Positive: 20


In [46]:
print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.87      0.92       233
           1       0.40      0.74      0.52        27

    accuracy                           0.86       260
   macro avg       0.68      0.81      0.72       260
weighted avg       0.91      0.86      0.88       260
 



In [24]:
# distillbert_predictor.save('../../model/first_generation_distilbert_base_uncased_model_10102020') # 256 MB

In [21]:
print("AUC roc score for distillbert model: ", roc_auc_score(distillbert_test_label,y_pred_distillbert))

AUC roc score for distillbert model:  0.6427102188579892
